In [2]:
# 1. Install Python libraries
!pip install openai -q

# 2. Install LaTeX (This takes about 2-3 minutes)
!apt-get update -q
!apt-get install -y texlive-latex-recommended texlive-fonts-recommended texlive-latex-extra -q

Hit:1 http://archive.ubuntu.com/ubuntu jammy InRelease
Get:2 http://archive.ubuntu.com/ubuntu jammy-updates InRelease [128 kB]
Get:3 http://archive.ubuntu.com/ubuntu jammy-backports InRelease [127 kB]
Get:4 http://security.ubuntu.com/ubuntu jammy-security InRelease [129 kB]
Get:5 https://cloud.r-project.org/bin/linux/ubuntu jammy-cran40/ InRelease [3,632 B]
Get:6 https://cli.github.com/packages stable InRelease [3,917 B]
Hit:7 https://ppa.launchpadcontent.net/deadsnakes/ppa/ubuntu jammy InRelease
Get:8 https://ppa.launchpadcontent.net/ubuntugis/ppa/ubuntu jammy InRelease [24.6 kB]
Get:9 https://r2u.stat.illinois.edu/ubuntu jammy InRelease [6,555 B]
Get:10 http://archive.ubuntu.com/ubuntu jammy-updates/restricted amd64 Packages [6,498 kB]
Get:11 http://archive.ubuntu.com/ubuntu jammy-updates/multiverse amd64 Packages [70.9 kB]
Get:12 http://archive.ubuntu.com/ubuntu jammy-updates/universe amd64 Packages [1,604 kB]
Get:13 http://archive.ubuntu.com/ubuntu jammy-updates/main amd64 Packages

In [4]:
!apt-get install texlive-latex-extra dvipng cm-super

Reading package lists... Done
Building dependency tree... Done
Reading state information... Done
texlive-latex-extra is already the newest version (2021.20220204-1).
The following additional packages will be installed:
  cm-super-minimal ghostscript pfb2t1c2pfb
Suggested packages:
  ghostscript-x
The following NEW packages will be installed:
  cm-super cm-super-minimal dvipng ghostscript pfb2t1c2pfb
0 upgraded, 5 newly installed, 0 to remove and 25 not upgraded.
Need to get 26.1 MB of archives.
After this operation, 60.3 MB of additional disk space will be used.
Get:1 http://archive.ubuntu.com/ubuntu jammy/universe amd64 cm-super-minimal all 0.3.4-17 [5,777 kB]
Get:2 http://archive.ubuntu.com/ubuntu jammy/universe amd64 pfb2t1c2pfb amd64 0.3-11 [9,342 B]
Get:3 http://archive.ubuntu.com/ubuntu jammy/universe amd64 cm-super all 0.3.4-17 [20.2 MB]
Get:4 http://archive.ubuntu.com/ubuntu jammy-updates/main amd64 ghostscript amd64 9.55.0~dfsg1-0ubuntu5.13 [49.4 kB]
Get:5 http://archive.ubunt

In [ ]:
import os
import re
import tarfile
import subprocess
import shutil
from openai import OpenAI
from google.colab import files

# --- CONFIG ---
client = OpenAI(api_key="", base_url="https://api.perplexity.ai")
MODEL_NAME = "sonar"

def clean_latex(text):
    """Escapes common LaTeX special characters that cause compilation errors."""
    # Escape underscores not in math mode, but be careful with existing commands
    text = re.sub(r'(?<!\\)_', r'\\_', text)
    # Ensure '&' is escaped unless it's a part of a known environment (rare in these snippets)
    text = re.sub(r'(?<!\\)&', r'\\&', text)
    return text

class ArchiveTool:
    @staticmethod
    def extract(archive_path):
        extract_path = archive_path.replace(".tar.gz", "_extracted").replace(".tar", "_extracted")
        if os.path.exists(extract_path):
            shutil.rmtree(extract_path)
        os.makedirs(extract_path)
        print(f"Extracting {archive_path}...")
        with tarfile.open(archive_path, "r:*") as tar:
            tar.extractall(path=extract_path)
        return extract_path

class WorkspaceTool:
    def __init__(self, directory):
        self.directory = directory
        self.images = [f for root, _, fs in os.walk(directory) for f in fs if f.lower().endswith(('.png', '.jpg', '.pdf'))]

    def find_main_tex(self):
        for root, _, files in os.walk(self.directory):
            for file in files:
                if file.endswith(".tex"):
                    path = os.path.join(root, file)
                    with open(path, 'r', encoding='utf-8', errors='ignore') as f:
                        content = f.read()
                        if "\\begin{document}" in content:
                            return path
        return None

    def extract_metadata(self, main_tex_path):
        with open(main_tex_path, 'r', encoding='utf-8', errors='ignore') as f:
            content = f.read()

        def get_field(tag):
            match = re.search(rf'\\{tag}\s*\{{(.*?)\}}', content, re.DOTALL)
            return match.group(1).strip() if match else "Untitled Research"

        return {
            "title": get_field("title"),
            "author": get_field("author"),
            "institute": get_field("institute")
        }

class DistillerTool:
    @staticmethod
    def process_section(title, content, available_images):
        prompt = f"""
        Task: Create a professional LaTeX Beamer frame for: {title}
        Raw Paper Content: {content[:3000]}
        Available Graphics: {available_images}

        Strict Formatting Constraints:
        1. REFRAMING: If a section is long, break it into bullet points. No point or paragraph can exceed 50 words.
        2. VERBATIM ALGORITHMS: If you see a proof, lemma, or algorithm, copy it EXACTLY as it is in the text. Do not summarize code.
        3. MARGINS: Use \\begin{{columns}} to ensure text and images do not touch the page edges.
        4. LAYOUT:
           - Left column (0.6\\textwidth): Key points.
           - Right column (0.4\\textwidth): A relevant image filename if applicable.
        5. OUTPUT: Return ONLY the LaTeX \\begin{{frame}}...\\end{{frame}} block. No markdown backticks.
        """
        try:
            response = client.chat.completions.create(
                model=MODEL_NAME,
                messages=[{"role": "user", "content": prompt}]
            )
            return response.choices[0].message.content
        except Exception as e:
            return f"\\begin{{frame}}{{{title}}}\\vfill\\centering Error processing slide content.\\vfill\\end{{frame}}"

def run_agent_colab(archive_file):
    work_dir = ArchiveTool.extract(archive_file)
    ws = WorkspaceTool(work_dir)
    main_tex = ws.find_main_tex()

    if not main_tex:
        return print("Error: Could not find main .tex file containing \\begin{document}.")

    metadata = ws.extract_metadata(main_tex)

    with open(main_tex, 'r', encoding='utf-8', errors='ignore') as f:
        full_text = f.read()
        clean_paper = re.sub(r'%.*', '', full_text)
        sections = re.findall(r'\\section\{(.+?)\}(.*?)(?=\\section|\\end\{document\})', clean_paper, re.DOTALL)

    distiller = DistillerTool()
    frames = []
    print(f"Generating slides for: {metadata['title']}...")

    for title, body in sections[:10]:
        clean_title = re.sub(r'\\.*\{.*\}', '', title).strip()
        print(f"-> Processing Section: {clean_title}")
        frame_code = distiller.process_section(clean_title, body.strip(), ws.images)
        frames.append(clean_latex(frame_code.replace("```latex", "").replace("```", "").strip()))

    output_tex = os.path.join(work_dir, "presentation.tex")
    with open(output_tex, 'w', encoding='utf-8') as f:
        f.write("\\documentclass[aspectratio=169, 10pt]{beamer}\n")
        f.write("\\usetheme{metropolis}\n")
        f.write("\\usepackage{graphicx, booktabs, amsmath, amssymb}\n")
        f.write("\\setbeamersize{text margin left=8mm, text margin right=8mm}\n")

        f.write(f"\\title{{{clean_latex(metadata['title'])}}}\n")
        f.write(f"\\author{{{clean_latex(metadata['author'])}}}\n")
        f.write(f"\\institute{{{clean_latex(metadata['institute'])}}}\n")
        f.write("\\date{\\today}\n\n\\begin{document}\n\n")

        f.write("\\begin{frame}\n  \\titlepage\n\\end{frame}\n\n")
        f.write("\\begin{frame}{Outline}\n  \\tableofcontents\n\\end{frame}\n\n")

        f.write("\n\n".join(frames))

        f.write("\n\n\\begin{frame}\n  \\centering\n  \\vfill\n  \\Huge \\textbf{Thank You!}\n  \\vfill\n  \\large Questions & Discussion\n  \\vfill\n\\end{frame}\n")
        f.write("\n\n\\end{document}")

    # Compilation
    os.chdir(work_dir)
    print("Compiling PDF...")
    subprocess.run(["pdflatex", "-interaction=nonstopmode", "presentation.tex"], capture_output=True)
    subprocess.run(["pdflatex", "-interaction=nonstopmode", "presentation.tex"], capture_output=True)

    if os.path.exists("presentation.pdf"):
        files.download("presentation.pdf")
        print("SUCCESS!")
    else:
        print("LaTeX Error: Check presentation.log for details.")

# Execution
uploaded = files.upload()
for filename in uploaded.keys():
    run_agent_colab(filename)

Saving Trust_proof_Learning_to_Collaborate.tar to Trust_proof_Learning_to_Collaborate.tar
Extracting Trust_proof_Learning_to_Collaborate.tar...
Generating slides for: Trust-proof Decentralized Learning to Collaborate...
-> Processing Section: Introduction


/tmp/ipython-input-499131197.py:31: DeprecationWarning: Python 3.14 will, by default, filter extracted tar archives and reject files or modify their metadata. Use the filter argument to control this behavior.
  tar.extractall(path=extract_path)


-> Processing Section: Related Work
-> Processing Section: Preliminaries and Model
-> Processing Section: Convergence theory of decentralised FL
-> Processing Section: Lower Bound for Trust-proof Ranking
-> Processing Section: Trust-proof Learning to Collaborate
-> Processing Section: Algorithm
-> Processing Section: Experiments
-> Processing Section: Committee Switching
-> Processing Section: Conclusion
Compiling PDF...


<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

SUCCESS!
